In [25]:
import re
import os
import json
import jieba 
import jieba.posseg as pseg

jieba.load_userdict('E:\论文\代码\数据处理\my_dict.txt')

<>:7: SyntaxWarning: invalid escape sequence '\m'
<>:7: SyntaxWarning: invalid escape sequence '\m'
C:\Users\wind\AppData\Local\Temp\ipykernel_16692\3790243778.py:7: SyntaxWarning: invalid escape sequence '\m'
  jieba.load_userdict('E:\论文\代码\数据处理\my_dict.txt')


In [26]:
# 读取数据文件
def readData(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    return lines

In [27]:
#获取数据文件数量
def get_file_count(directory_path):
    try:
        file_count = 0
        with os.scandir(directory_path) as entries:
            for entry in entries:
                if entry.is_file():
                    file_count += 1
        return file_count
    except FileNotFoundError:
        print(f"目录不存在: {directory_path}")
        return 0
    except PermissionError:
        print(f"没有权限访问目录: {directory_path}")
        return 0

In [28]:
# 情绪字典
def is_emotion_word(word):
    emotion_words = {
        '高兴', '快乐', '开心', '愉快', '兴奋', '激动', '幸福',
        '悲伤', '难过', '伤心', '痛苦', '悲哀', '沮丧', '失望',
        '愤怒', '生气', '气愤', '恼怒', '暴躁', '愤怒',
        '害怕', '恐惧', '惊恐', '慌张', '紧张', '焦虑',
        '惊讶', '惊奇', '吃惊', '意外', '震惊',
        '讨厌', '厌恶', '反感', '憎恶', '嫌弃',
        '喜欢', '爱', '欣赏', '迷恋', '陶醉'
    }
    return word in emotion_words


In [29]:
# 找到对话中的人名以及形容词
def find_name_adjective(dialogue):
    text_without_quotes = re.sub(r'"[^"]*', '', dialogue)
    words = pseg.cut(text_without_quotes)
    person_flags = {'nr', 'nrt'}
    adjective_flags = {'a'}
    persons = []
    adjectives = []
    for word, flag in words:
        if flag in person_flags:
            persons.append(word)
        elif flag in adjective_flags and is_emotion_word(word):
            adjectives.append(word)
    return persons, adjectives

In [30]:
def extract_TangSan_dialogues(line):
    train_data = []
    patterns = [
        r'.*?["“](.*?)["”]',
        # r'唐三.*?["“](.*?)["”]',
        # r'^(?!.*唐三.*["“].*["”]).*["“](.*?)["”](?!唐三)'
    ]
    dialogues = re.findall(patterns[0], line)
    if dialogues:
        persons, adjectives = find_name_adjective(line)
        train_data.append({
            'character': persons,
            'emotion': adjectives,
            'dialogue': dialogues
        })
    return train_data

In [31]:
def export_to_conversation_format(data, output_path):
    formatted_data = []
    for i in range(1, len(data)):
        item = data[i]
        last_speak_character = data[i]['character']
        # 构建对话上下文
        personSystem = "你是唐三，前世为唐门外门弟子，因偷学内门绝技《玄天宝录》而自杀谢罪，穿越至斗罗大陆。你是万年不遇的双生武魂拥有者（昊天锤、蓝银皇），未来的海神与修罗神双神位继承者。你性格沉稳坚毅、睿智冷静、极其重情重义，尤其深爱小舞，愿为她付出一切。你秉持‘人不犯我，我不犯人；人若犯我，步步逼人’的准则。你精通唐门绝学（玄玉手、紫极魔瞳、暗器百解等），擅长在战斗中使用智慧和技巧。"
        # 如果姓名列表内有唐三两字且出现的频率是最高的才创建对话格式
        if '唐三' in item['character']:
            # 创建对话格式
            messages = {
                "messages": [
                    {
                    "system": personSystem,
                    "input": data[i - 1]['dialogue'][0],
                    "output": data[i]['dialogue'][0]
                    }
                ]
            }
            # 如果上一次说话的人也是唐三则把新的一段对话合并
            if last_speak_character and '唐三' in last_speak_character:
                formatted_data[-1]['messages'][0]['output'] += " " + messages['messages'][0]['output']
            else:
                formatted_data.append(messages)

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(formatted_data, f, ensure_ascii=False, indent=2)
    print(f"对话格式数据已导出到: {output_path}")

In [32]:
if __name__ == "__main__":
    train_data = []
    train_data_path = r"E:\论文\代码\数据处理\NonEmptyData"
    output_path = r"E:\论文\代码\数据处理\output"
    #获取文件数量
    file_count = get_file_count(train_data_path)
    for i in range(1, file_count + 1):
        file_path = os.path.join(train_data_path, f"train_{i}.txt")
        lines = readData(file_path)
        for line in lines:
            train_data.extend(extract_TangSan_dialogues(line))
    export_to_conversation_format(train_data, os.path.join(output_path, "conversations.json"))
    
    # 打印前几条数据作为示例
    print("\n前3条数据示例:")
    for i, item in enumerate(train_data[:3]):
        print(f"数据 {i+1}:")
        print(f"  人物: {item['character']}")
        print(f"  情感词: {item['emotion']}")
        print(f"  对话: {item['dialogue']}")

IndexError: list index out of range